In [ ]:

# ------------------------------------------
# 2. Import Libraries
# ------------------------------------------
# Importing required Python libraries:
# - numpy and pandas for data manipulation
# - datetime for handling timestamps
# - pathway for streaming data workflows
# - bokeh for visualizing time-series data interactively in notebooks
import numpy as np
import pandas as pd
import datetime
import pathway as pw
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import ColumnDataSource
output_notebook()  # Enables bokeh plots to display inside the notebook



In [ ]:
import numpy as np
import pandas as pd
import datetime
import pathway as pw
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import ColumnDataSource
output_notebook()


In [ ]:
# ------------------------------------------
# 3. Load & Preprocess Dataset
# ------------------------------------------
# Load the dataset from CSV file
df = pd.read_csv('dataset.csv')

# Combine 'LastUpdatedDate' and 'LastUpdatedTime' into one timestamp column
df['Timestamp'] = pd.to_datetime(df['LastUpdatedDate'] + ' ' + df['LastUpdatedTime'], format='%d-%m-%Y %H:%M:%S')

# Sort data chronologically and reset index for clean processing
df = df.sort_values('Timestamp').reset_index(drop=True)

# Save the cleaned dataset to a new CSV file
df.to_csv("parking_stream_clean.csv", index=False)

# Show first few rows of the cleaned dataset
df.head()

,ID,SystemCodeNumber,Capacity,Latitude,Longitude,Occupancy,VehicleType,TrafficConditionNearby,QueueLength,IsSpecialDay,LastUpdatedDate,LastUpdatedTime,Timestamp
0,0,BHMBCCMKT01,577,26.144536,91.736172,61,car,low,1,0,04-10-2016,07:59:00,2016-10-04 07:59:00
1,5248,BHMNCPHST01,1200,26.140014,91.731000,237,bike,low,2,0,04-10-2016,07:59:00,2016-10-04 07:59:00
2,3936,BHMMBMMBX01,687,20.000035,78.000003,264,car,low,2,0,04-10-2016,07:59:00,2016-10-04 07:59:00
3,6560,BHMNCPNST01,485,26.140048,91.730972,249,car,low,2,0,04-10-2016,07:59:00,2016-10-04 07:59:00
4,17056,Shopping,1920,26.150504,91.733531,614,cycle,low,2,0,04-10-2016,07:59:00,2016-10-04 07:59:00


In [ ]:
# ------------------------------------------
# 4. Model 1: Baseline Linear Pricing
# ------------------------------------------
# A simple pricing model that increases price based on occupancy ratio.
# 'alpha' is a tuning parameter controlling price sensitivity to occupancy.
def model_1_price(prev_price, occupancy, capacity, alpha=0.05):
    return prev_price + alpha * (occupancy / capacity)


In [ ]:
#  5. Model 2: Demand-Based Pricing
# ------------------------------------------
# Calculates demand using multiple factors:
# - Occupancy level
# - Queue length at parking
# - Traffic conditions nearby
# - Whether the day is special (e.g., holiday)
# - Type of vehicle (car, bike, truck)
def calculate_demand(row, weights):
    traffic_map = {'low': 0.3, 'medium': 0.6, 'high': 1.0}
    vehicle_map = {'car': 1.0, 'bike': 0.6, 'truck': 1.5}

    demand = (
        weights['occ'] * (row['Occupancy'] / row['Capacity']) +
        weights['queue'] * row['QueueLength'] -
        weights['traffic'] * traffic_map.get(row['TrafficConditionNearby'], 0.5) +
        weights['special'] * row['IsSpecialDay'] +
        weights['vehicle'] * vehicle_map.get(row['VehicleType'], 1.0)
    )
    return demand

In [ ]:

# Adjusts base price based on calculated demand.
# 'lambda_' controls how sensitive price is to demand.
# Price is bounded to avoid extreme values.
def model_2_price(base_price, demand, demand_bounds=(0, 2), lambda_=0.3):
    normalized_demand = max(0, min(1, (demand - demand_bounds[0]) / (demand_bounds[1] - demand_bounds[0])))
    price = base_price * (1 + lambda_ * normalized_demand)
    return min(max(price, base_price * 0.5), base_price * 2)


In [ ]:
# 6. Apply Models to Data
# ------------------------------------------
# Set weights for demand factors in Model 2
weights = {'occ': 1.2, 'queue': 0.5, 'traffic': 0.8, 'special': 0.3, 'vehicle': 0.4}

# Initialize base price and container lists to hold results
base_price = 10.0
prev_price = base_price
prices_model1 = []
prices_model2 = []


In [ ]:
# Iterate over each row (each timestamped record)
for _, row in df.iterrows():
    # Model 1: Calculate next price based on previous price and occupancy
    price1 = model_1_price(prev_price, row['Occupancy'], row['Capacity'])
    prices_model1.append(price1)

    # Model 2: Calculate demand and then price based on that demand
    demand = calculate_demand(row, weights)
    price2 = model_2_price(base_price, demand)
    prices_model2.append(price2)

    # Update previous price for next Model 1 calculation
    prev_price = price1


In [ ]:
# Add calculated prices to the DataFrame
df['Price_Model1'] = prices_model1
df['Price_Model2'] = prices_model2

In [ ]:
# 7. Visualize with Bokeh
# ------------------------------------------
# Create a data source for bokeh plotting
source = ColumnDataSource(df)

# Set up a line plot with datetime x-axis to show price changes over time
p = figure(x_axis_type='datetime', title="Parking Price Over Time", width=800, height=400)

# Add price trends for both models
p.line(x='Timestamp', y='Price_Model1', source=source, color='blue', legend_label='Model 1 Price')
p.line(x='Timestamp', y='Price_Model2', source=source, color='green', legend_label='Model 2 Price')

# Add legend and show the plot
p.legend.location = "top_left"
show(p)